# Development Scratch

This notebook contains ideas/scratch/notes for development

In [6]:
import os
import sys

PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, PWD)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
django.setup()

In [10]:
import datetime
import django
import pandas as pd

from availabilities.models import *
from availabilities.utils import check_availabilities
from availabilities.utils import get_availabilities_for_participants
from events.models import *
from events.utils import get_all_event_participants
from collections import defaultdict

# os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
# django.setup()

In [11]:
FEB_3_9AM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=9,
    tzinfo=datetime.timezone.utc
)

FEB_3_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

FEB_3_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_3_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

FEB_4_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=4, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_4_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=4, 
    hour=17,
    tzinfo=datetime.timezone.utc
)


FEB_5_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=5, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_5_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=5, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

In [28]:
# Generates test data: Not available 2022/2/3 9-5PM
andrew_defaults = {
    "date_time_start": FEB_3_9AM,
    "date_time_end": FEB_3_5PM,
    "is_available": False
}

jessie_defaults = {
    "date_time_start": FEB_4_2PM,
    "date_time_end": FEB_4_5PM,
    "is_available": False
}

andrew, __ = User.objects.get_or_create(username="Andrew")
jessie, __ = User.objects.get_or_create(username="Jessie")

availability, __ = DateAvailability.objects.update_or_create(
    user=andrew,
    category="hockey_team",
    defaults=andrew_defaults
)

availability, __ = DateAvailability.objects.update_or_create(
    user=jessie,
    category="social_stuff",
    defaults=jessie_defaults
)

# Generates event
event_defaults = {
    "members": []
}
event, __ = Event.objects.get_or_create(
    name="Hockey Party",
)
event.save()

# Connects users to event
UserEvent.objects.update_or_create(
    user=andrew,
    event=event,
    defaults={
        "is_host": True,
        "is_required": True
    }
)
UserEvent.objects.update_or_create(
    user=jessie,
    event=event,
    defaults={
        "is_host": False,
        "is_required": True
    }
)

# Generates proposed times
proposed_times = [
    (FEB_3_2PM, FEB_3_5PM),
    (FEB_4_2PM, FEB_4_5PM),
    (FEB_5_2PM, FEB_5_5PM),
]
for proposed_start, proposed_end in proposed_times:
    EventTime.objects.get_or_create(
        event=event,
        date_time_start=proposed_start,
        date_time_end=proposed_end,
    )

# Peeks
event_time_df = pd.DataFrame(EventTime.objects.filter(event=event).values())
event_time_df

,id,event_id,date_time_start,date_time_end
0,1,1,2022-02-03 14:00:00+00:00,2022-02-03 17:00:00+00:00
1,2,1,2022-02-04 14:00:00+00:00,2022-02-04 17:00:00+00:00
2,3,1,2022-02-05 14:00:00+00:00,2022-02-05 17:00:00+00:00


In [ ]:
def process_suggested_date(user, event, suggested_event_time):
    """
    Compares suggested_event_time against all members in event
    """
    participants = get_all_event_participants(event)
    availability_dict, times_that_work = check_availabilities(event)
    
    time_works = suggested_event_time.id in times_that_work
    available_participants = availability_dict[suggested_event_time.id]
    unavailable_participants = list(set(participants) - set(available_participants))
        
    return time_works, unavailable_participants
    

user = andrew
suggested_event_time = EventTime.objects.get(
    event_id=event.id,
    date_time_start=FEB_5_2PM
)

process_suggested_date(user, event, suggested_event_time)

In [34]:
user_ids = list(Event.objects.filter(id=event.id).values_list("users__user_id", flat=True))
users = list(User.objects.filter(id__in=user_ids))

[<User: Andrew>, <User: Jessie>]

In [22]:
pd.DataFrame(UserEvent.objects.values())

,id,state,user_id,event_id,is_required,is_host
0,1,None,1,1,True,True
1,2,None,1,1,True,True
2,3,None,2,1,True,False


In [68]:
event.objects.select_related("users__user")

AttributeError: Manager isn't accessible via Event instances

In [70]:
event.users.

AttributeError: 'Event' object has no attribute 'values'

In [91]:
event.users.select_related("event").all()

<QuerySet [<UserEvent: UserEvent object (4)>, <UserEvent: UserEvent object (5)>]>